In [1]:
from premise import *
import bw2data
from datapackage import Package
bw2data.projects.set_current("ei310")

In [2]:
sps = Package("../datapackage.json")
scenarios = [
    {"model": "image", "pathway": "SSP2-RCP26", "year": 2020, "external scenarios": [{"scenario": "SPS1", "data": sps}]},
    #{"model": "image", "pathway": "SSP2-RCP26", "year": 2022, "external scenarios": [{"scenario": "SPS1", "data": sps}]},
    #{"model": "image", "pathway": "SSP2-RCP26", "year": 2030, "external scenarios": [{"scenario": "SPS1", "data": sps}]},
    #{"model": "image", "pathway": "SSP2-RCP26", "year": 2040, "external scenarios": [{"scenario": "SPS1", "data": sps}]},
    #{"model": "image", "pathway": "SSP2-RCP26", "year": 2050, "external scenarios": [{"scenario": "SPS1", "data": sps}]},
]

ndb = NewDatabase(
        scenarios=scenarios,
        source_db="ecoinvent 3.9.1 cutoff",
        source_version="3.9.1",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        use_multiprocessing=False,
        use_absolute_efficiency=True
        
)

premise v.(2, 1, 0, 'dev4')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-----

In [3]:
ndb.update()

Processing scenarios:   0%|                     | 0/1 [00:00<?, ?it/s]

Extracted 1 worksheets in 0.08 seconds
The following datasets will be duplicated:
+----------------------------------------------------+----------------------------------------------------+
|                        Name                        |                 Reference product                  |
+----------------------------------------------------+----------------------------------------------------+
| heat and power co-generation, natural gas, 1MW ele |             electricity, high voltage              |
| heat and power co-generation, natural gas, 160kW e |              electricity, low voltage              |
| electricity production, from incineration of plast | electricity, from incineration of plastic waste mi |
| electricity production, at co-generation plastic w | electricity, from plastic waste mixture, municipal |
| electricity production, from biowaste, municipal i | electricity, from biowaste, municipal incineration |
| electricity production, at co-generation biowaste- |

Processing scenarios: 100%|████████████| 1/1 [01:48<00:00, 108.40s/it]

Done!



In [4]:
ndb.write_db_to_brightway([
    "test stem 2020",
    #"test stem 2050"
])

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.
Database test stem 2020 already exists: it will be overwritten.
Vacuuming database 


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:43


Title: Writing activities to SQLite3 database:
  Started: 05/03/2024 10:34:55
  Finished: 05/03/2024 10:35:38
  Total time elapsed: 00:00:43
  CPU %: 94.90
  Memory %: 17.53
Created database: test stem 2020
Generate scenario report.
Report saved under /Users/romain/Github/sweet_sure-2050-switzerland/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/Github/sweet_sure-2050-switzerland/dev.


In [1]:
from premise import __file__
__file__

'/Users/romain/Github/premise/premise/__init__.py'

In [1]:
from premise import *
import bw2data
from datapackage import Package
bw2data.projects.set_current("ei310")

In [2]:
#clear_inventory_cache()
sps = Package("../datapackage.json")
scenarios=[
    {"model": "remind", "pathway": "SSP2-NPi", "external scenarios": [{"scenario": "SPS4", "data": sps}]},
    #{"model": "remind", "pathway": "SSP2-PkBudg1150", "external scenarios": [{"scenario": "SPS4", "data": sps}]},
    #{"model": "remind", "pathway": "SSP2-PkBudg500", "external scenarios": [{"scenario": "SPS4", "data": sps}]},
    #{"model": "remind", "pathway": "SSP2-NPi", "external scenarios": [{"scenario": "SPS1", "data": sps}]},
    #{"model": "remind", "pathway": "SSP2-PkBudg1150", "external scenarios": [{"scenario": "SPS1", "data": sps}]},
    #{"model": "remind", "pathway": "SSP2-PkBudg500", "external scenarios": [{"scenario": "SPS1", "data": sps}]},
]

for scenario in scenarios:
    ndb = PathwaysDataPackage(
        scenarios=[
            scenario,
        ],
        years=[
            2020, 
            #2050
        ],
        #years=[2020, 2025, 2030, 2035, 2040, 2045, 2050],
        source_db="ecoinvent-3.10-cutoff", # <-- name of the database in the BW2 project. Must be a string.
        source_version="3.10", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
        key="tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=",
        use_absolute_efficiency=True,
        biosphere_name="ecoinvent-3.10-biosphere"
    )
    
    ndb.create_datapackage(
        name="image-SSP2-stem-SPS1",
        contributors=[
            {"name": "Romain",
            "email": "r_s at me.com",}
        ],
    )

premise v.(2, 1, 1, 'dev0')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-----

Processing scenarios:   0%|                     | 0/1 [00:00<?, ?it/s]

Extracted 1 worksheets in 0.07 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
The following datasets will be duplicated:
+----------------------------------------------------+----------------------------------------------------+
|                        Name                        |                 Reference product                  |
+----------------------------------------------------+----------------------------------------------------+
| heat and power co-generation, natural gas, 1MW ele |             electricity, high voltage              |
| heat and power co-generation, natural gas, 160kW e |              electricity, low voltage              |
| electricity production, from incineration of plast | electricity, from incineration of plastic waste mi |
| electricity production, at co-generation plastic w | electricity, from plastic waste mixture, municipal |
| electricity production, from biowaste, municipal i | electricity, from biowaste, munici

Processing scenarios: 100%|████████████| 1/1 [01:47<00:00, 107.49s/it]


Done!

Extracted 1 worksheets in 0.03 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Remove uncertainty data.
Write new database(s) to matrix.
Running all checks...
Minor anomalies found: check the change report.
Matrices saved in /Users/romain/Github/sweet_sure-2050-switzerland/dev/pathways/inventories/remind/SSP2-NPi-SPS4/2020.
Generate scenario report.
Report saved under /Users/romain/Github/sweet_sure-2050-switzerland/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/Github/sweet_sure-2050-switzerland/dev.
Leaving out SE|Electricity|Gas|++|Gas Turbine from Gas ST
Leaving out SE|Liquids|+|Hydrogen from diesel, synthetic, from electrolysis
Leaving out SE|Liquids|Fossil|Coal|+|w/o CC from diesel, synthetic, from coal
Leaving out SE|Liquids|Fossil|Coal

In [3]:
for ds in ndb.datapackage.scenarios[0]["database"]:
    for e in ds["exchanges"]:
        if e["name"] == "2-Methylpentane":
            print(ds["name"])

petrol, burned in passenger car
petrol, burned in passenger car
petrol, burned in passenger car
petrol, burned in passenger car
petrol, burned in passenger car
petrol, burned in passenger car
petrol, burned in passenger car
petrol, burned in passenger car
petrol, burned in passenger car
petrol, burned in passenger car
petrol, burned in passenger car
petrol, burned in passenger car
petrol, burned in passenger car


In [8]:
from premise.activity_maps import InventorySet
i = InventorySet(ndb.datapackage.database, "3.10", "image")

In [9]:
d = i.generate_metals_activities_map()

In [10]:
from pprint import pprint
pprint(d)

{'AEL': {'electrolyzer production, 1MWe, AEC, Stack'},
 'CIGS': {'photovoltaic laminate production, CIS',
          'photovoltaic laminate, CIS, at plant'},
 'CSP Tower - PB': {'power block installation, solar tower power plant, 20 MW'},
 'CdTe': {'photovoltaic laminate production, CdTe',
          'photovoltaic laminate, CdTe, First Solar Series 4, at plant',
          'photovoltaic laminate, CdTe, First Solar Series 6, at plant'},
 'EV': {'electric motor production, for electric scooter',
        'electric motor production, vehicle',
        'electric motor production, vehicle (electric powertrain)'},
 'HTEL - Yttrium': {'electrolyzer production, 1MWe, SOEC, Stack'},
 'ICEV': {'frame, blanks and saddle, for lorry',
          'passenger car production, diesel',
          'passenger car production, petrol/natural gas'},
 'LFP': {'battery cell production, Li-ion, LFP'},
 'LMO': {'cathode production, LiMn2O4, for Li-ion battery'},
 'Lead_Acid': {'battery production, lead acid, rechargeab

In [3]:
ndb.create_datapackage(
    name="image-SSP2-stem-SPS1",
    contributors=[
        {"name": "Romain",
        "email": "r_s at me.com",}
    ],
)

Processing scenarios:   0%|                     | 0/8 [00:00<?, ?it/s]

Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal rare earth carbonate concentrate not found in alternative names. Skipping transport.
Metal rare earth carbonat

Processing scenarios:  12%|█▌          | 1/8 [01:47<12:35, 107.98s/it]

Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal rare earth carbonate concentrate not found in alternative names. Skipping transport.
Metal rare earth carbonat

Processing scenarios:  25%|███         | 2/8 [03:43<11:16, 112.71s/it]

...

Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal rare earth carbonate concentrate not found in alternative names. Skipping transport.
Metal rare earth carbonat

Processing scenarios:  88%|██████████▌ | 7/8 [13:30<01:59, 119.74s/it]

Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal rare earth carbonate concentrate not found in alternative names. Skipping transport.
Metal rare earth carbonat

Processing scenarios: 100%|████████████| 8/8 [15:32<00:00, 116.54s/it]


Done!

Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Write new database(s) to matrix.
Running all checks...
Minor anomalies found: check the change report.
Matrices saved in /Users/romain/Github/sweet_sure-2050-switzerland/dev/pathways/inventories/image/SSP2-RCP26-SPS1/2020.
Running all checks...
Minor anomalies found: check the change report.
Matrices saved in /Users/romain/Github/sweet_sure-2050-switzerland/dev/pathways/inventories/image/SSP2-RCP26-SPS1/2022.
Running all checks...
Minor anomalies found: check the change report

In [4]:
from pathways import Pathways
p = Pathways(datapackage="image-SSP2-stem-SPS1.zip", debug=True)

Invalid datapackage: Descriptor validation error: {'path': 'mapping/mapping.yaml', 'profile': 'data-resource', 'name': 'mapping', 'format': 'yaml', 'mediatype': 'text/yaml', 'encoding': 'utf-8'} is not valid under any of the given schemas at "resources/33" in descriptor and at "properties/resources/items/oneOf" in profile
Invalid datapackage: Descriptor validation error: 'data-resource' is not one of ['tabular-data-resource'] at "resources/33/profile" in descriptor and at "properties/resources/items/properties/profile/enum" in profile


In [5]:
elec = [
        "hydro_run_of_river",
        "hydro (reservoir)",
        "hydro (pumped storage)",
        "nuclear (boiling water)",
        "nuclear (pressure water)",
        "oil (peak devices)",
        "oil (DH CHP)",
        "oil (industry CHP)",
        "gas (CCGT)",
        "gas (OCGT)",
        "gas (CCGT CCS)",
        "gas (CHP DH)",
        "gas (fuel cell DH)",
        "gas (CHP industry)",
        "gas (CHP CCS industry)",
        "gas (CHP fuel cell reformer)",
        "gas (CHP single family)",
        "gas (single family CHP fuel cell)",
        "gas (multi family CHP)",
        "gas (multi family CHP fuel cell)",
        "biogas (services CHP)",
        "gas (services CHP)",
        "gas (services CHP fuel cell)",
        "waste incineration, fossil (CHP)",
        "waste incineration, fossil (industrial)",
        "waste incineration, fossil (CHP, CCS)",
        "waste incineration, fossil (industrial, CCS)",
        "waste incineration, non-fossil (CHP)",
        "waste incineration, non-fossil (industrial)",
        "waste incineration, non-fossil (CHP, CCS)",
        "waste incineration, non-fossil (industrial, CCS)",
        "pv (large scale)",
        "pv (industry)",
        "pv (services)",
        "pv residential (multi family)",
        "pv residential (single family)",
        "wind",
        "geothermal",
        "hydrogen fuel cell (DH CHP)",
        "hydrogen fuel cell (industrial CHP)",
        "hydrogen fuel cell (services CHP)",
        "biomass (DH)",
        "biomass (CHP)",
        "biomass (CCS, BECCS)",
        "biomass (CCS, industrial)",
        "battery (large)",
        "battery (medium)",
        "battery (industry)",
        "battery (services)",
        "battery (single family)",
        "battery (multi family)",
        "CAES",
        "import",
    ]

In [6]:
import numpy as np
p.calculate(
    methods=[
        #'IPCC 2021 - climate change - GWP 100a, incl. H',
        #'IPCC 2021 - climate change - GWP 100a, incl. H and bio CO2',
        'EF v3.1 EN15804 - climate change - global warming potential (GWP100)',
        'ReCiPe 2016 v1.03, endpoint (I) - total: ecosystem quality - ecosystem quality',
        'EF v3.1 - material resources: metals/minerals - abiotic depletion potential (ADP): elements (ultimate reserves)',
        'Ecological Scarcity 2021 - total - UBP',
        'ReCiPe 2016 v1.03, endpoint (E) - total: human health - human health',
        'selected LCI results - resource - land occupation',
        'Cumulative Energy Demand (CED) - energy resources: renewable - energy content (HHV)',
        'Cumulative Energy Demand (CED) - energy resources: non-renewable - energy content (HHV)',
    ]  + [m for m in p.lcia_methods if "RELICS" in m],
    regions=["CH",],
    scenarios=[
        "SSP2-RCP26-SPS1",
        #"SSP2-RCP19",
    ],
    variables=[v for v in p.scenarios.coords["variables"].values if v.startswith("FE")],
    years=[
        2020,
        2022,
        2025,
        2030,
        2035,
        2040,
        2045,
        2050
    ],
    multiprocessing=False,
    use_distributions=0,
)

Calculating LCA results for image...
--- Calculating LCA results for SSP2-RCP26-SPS1...
------ Calculating LCA results for 2020...


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


------ Calculating LCA results for 2022...


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


------ Calculating LCA results for 2025...


...

0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


------ Calculating LCA results for 2045...


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


------ Calculating LCA results for 2050...


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


In [7]:
p.lca_results = p.lca_results.sum(dim="act_category")

In [8]:
arr = p.lca_results.interp(
    year=range(p.lca_results.coords["year"].values.min(), p.lca_results.coords["year"].values.max() + 1)
)

In [9]:
df = arr.to_dataframe("value")

In [10]:
df = df[df["value"]!=0.0]
df = df[~df["value"].isnull()]
print(len(df))
df=df.reset_index()
print(df["year"].unique())

7412401
[2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031 2032 2033
 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045 2046 2047
 2048 2049 2050]


In [11]:
from premise.geomap import Geomap
geo = Geomap("image")

In [12]:
EU = geo.iam_to_ecoinvent_location("WEU")
EU.remove("CH")
len(EU)

70

In [13]:
df.loc[
    df["location"].isin(EU), "location"
] = "EU wo CH"

In [14]:
df.loc[
    ~df["location"].isin(["CH", "EU wo CH"]), "location"
] = "RoW"

d = {
    loc: geo.ecoinvent_to_iam_location(loc)
    for loc in df["location"].unique()
}
df["region from"] = df["location"].map(d)
df = df.drop('location', axis=1)

In [15]:
df = df.groupby([
    'variable', 'year', 'region', 'model', 'scenario', 'impact_category', 'location', #"act_category"
]).sum().reset_index()
print(len(df))

226755


In [16]:
df["impact_category"] = df["impact_category"].str.replace("RELICS - metals extraction - ", "")
df["impact_category"] = df["impact_category"].str.replace("EF v3.1 - climate change - global warming potential (GWP100)", "climate change")
df["impact_category"] = df["impact_category"].str.replace("EF v3.1 EN15804 - climate change - global warming potential (GWP100)", "climate change")
df["impact_category"] = df["impact_category"].str.replace("selected LCI results - resource - land occupation", "land occupation")
df["impact_category"] = df["impact_category"].str.replace("EF v3.1 - material resources: metals/minerals - abiotic depletion potential (ADP): elements (ultimate reserves)", "natural resources (minerals and metals)")
df["impact_category"] = df["impact_category"].str.replace("IPCC 2021 - climate change - GWP 100a, incl. H and bio CO2", "climate change")
df["impact_category"] = df["impact_category"].str.replace("Cumulative Energy Demand (CED) - energy resources: renewable - energy content (HHV)", "renewable primary energy")
df["impact_category"] = df["impact_category"].str.replace("Cumulative Energy Demand (CED) - energy resources: non-renewable - energy content (HHV)", "non-renewable primary energy")
df["impact_category"] = df["impact_category"].str.replace("ReCiPe 2016 v1.03, endpoint (I) - total: ecosystem quality - ecosystem quality", "ecosystem impact")
df["impact_category"] = df["impact_category"].str.replace("ReCiPe 2016 v1.03, endpoint (E) - total: human health - human health", "human health")
df["impact_category"] = df["impact_category"].str.replace("Ecological Scarcity 2021 - total - UBP", "ecological scarcity")

In [17]:
units = {
    "climate change": "kg CO2-eq.",
    "land occupation": "square meter-year",
    "natural resources (minerals and metals)": "kg Sb-eq.",
    "renewable primary energy": "MJ (HHV)",
    "non-renewable primary energy": "MJ (HHV)",
    "ecosystem impact": "species-year lost",
    "human health": "DALY",
    "ecological scarcity": "UBP pt"
}
df["unit"] = df["impact_category"].map(units)

In [18]:
df["impact_category"].unique()

array(['non-renewable primary energy', 'renewable primary energy',
       'natural resources (minerals and metals)', 'climate change',
       'ecological scarcity', 'Aluminium', 'Antimony', 'Beryllium',
       'Boron', 'Cadmium', 'Cerium', 'Chromium', 'Cobalt', 'Copper',
       'Dysprosium', 'Erbium', 'Europium', 'Gadolinium', 'Gallium',
       'Gold', 'Graphite', 'Hafnium', 'Indium', 'Lanthanum', 'Lead',
       'Lithium', 'Magnesium', 'Manganese', 'Molybdenum', 'Neodymium',
       'Nickel', 'Niobium', 'Palladium', 'Platinum', 'Potassium',
       'Praseodymium', 'Rhenium', 'Rhodium', 'Ruthenium', 'Samarium',
       'Scandium', 'Selenium', 'Silicon', 'Silver', 'Strontium', 'Sulfur',
       'Tantalum', 'Tellurium', 'Terbium', 'Tin', 'Titanium', 'Tungsten',
       'Vanadium', 'Ytterbium', 'Yttrium', 'Zinc', 'Zirconium',
       'human health', 'ecosystem impact', 'land occupation', 'Iridium'],
      dtype=object)

In [19]:
df.head()

,variable,year,region,model,scenario,impact_category,location,value,unit
0,FE_bus_diesel,2020,CH,image,SSP2-RCP26-SPS1,non-renewable primary energy,EU wo CH,1.718487e+08,MJ (HHV)
1,FE_bus_diesel,2020,CH,image,SSP2-RCP26-SPS1,non-renewable primary energy,RoW,8.534095e+08,MJ (HHV)
2,FE_bus_diesel,2020,CH,image,SSP2-RCP26-SPS1,renewable primary energy,CH,6.585922e+05,MJ (HHV)
3,FE_bus_diesel,2020,CH,image,SSP2-RCP26-SPS1,renewable primary energy,EU wo CH,1.627056e+07,MJ (HHV)
4,FE_bus_diesel,2020,CH,image,SSP2-RCP26-SPS1,renewable primary energy,RoW,9.662686e+06,MJ (HHV)


In [20]:
df.loc[df["unit"].isnull(), "unit"] = "kg"

In [21]:
from pivottablejs import pivot_ui
from IPython.display import HTML
pivot_ui(df, outfile_path='pivottable_ch.html')

In [22]:
pivot = df.pivot_table(index=['variable', 'region', 'model', 'scenario', 'impact_category', 'location', 'unit'], 
                          columns='year', 
                          values='value', 
                          aggfunc='sum').reset_index()
pivot.fillna(0, inplace=True)

In [23]:
scenarios = p.scenarios.sel(region="CH", pathway='SSP2-RCP26-SPS1', model="image").astype(float).interp(
    year=range(p.scenarios.coords["year"].values.min(), p.scenarios.coords["year"].values.max() + 1)
)

In [24]:
df_scenarios = scenarios.to_dataframe("value").reset_index().pivot_table(
    index=['variables', 'region', 'model', 'pathway', ], 
                          columns='year', 
                          values='value', 
                          aggfunc='sum'
)

In [25]:
df_scenarios["unit"] = "PJ"
df_scenarios["impact_category"] = "Production"

In [26]:
df_scenarios = df_scenarios.reset_index()

In [27]:
df_scenarios.rename(columns={'variables':'variable'}, inplace=True)
df_scenarios.rename(columns={'pathway':'scenario'}, inplace=True)

In [28]:
import pandas as pd
df_final = pd.concat([pivot, df_scenarios], ignore_index=True)

In [29]:
df_final.loc[df_final["location"].isnull(), "location"] = "CH"

In [30]:
df_final.to_excel("indicators_export_SPS1.xlsx")

In [5]:
!pip install pyyaml

In [6]:
# Adjusting the YAML loader to handle Python-specific objects (tuples)
import yaml
from yaml import SafeLoader
from yaml.constructor import SafeConstructor

# Define a constructor for Python tuples
def construct_python_tuple(loader, node):
    return tuple(loader.construct_sequence(node))

# Add the constructor to the SafeLoader
SafeLoader.add_constructor('tag:yaml.org,2002:python/tuple', construct_python_tuple)

file_path = "/Users/romain/Github/pathways/pathways/data/activities_classifications.yaml"

# Now, let's try loading the YAML file again with the adjusted loader
with open(file_path, 'r') as file:
    data_adjusted = yaml.load(file, Loader=SafeLoader)

# Display the first few items to understand its structure with adjustments
list(data_adjusted.items())[:5]

[(('1,1-difluoroethane production, HFC-152a',
   '1,1-difluoroethane, HFC-152a',
   'kilogram',
   'RoW'),
  'Manufacture of basic chemicals'),
 (('1,1-difluoroethane production, HFC-152a',
   '1,1-difluoroethane, HFC-152a',
   'kilogram',
   'US'),
  'Manufacture of basic chemicals'),
 (('1,1-dimethylcyclopentane to generic market for solvent, organic',
   'solvent, organic',
   'kilogram',
   'GLO'),
  'Manufacture of chemicals and chemical products'),
 (('1-methoxy-2-propanol production',
   '1-methoxy-2-propanol',
   'kilogram',
   'GLO'),
  'Manufacture of basic chemicals'),
 (('1-methylcyclopropene production',
   '1-methylcyclopropene',
   'kilogram',
   'GLO'),
  'Manufacture of basic chemicals')]

In [7]:
# Modify the data by removing the last item from each tuple key
modified_data = {(key[:-1]): value for key, value in data_adjusted.items()}

# Define the new file path for the modified YAML
modified_file_path = 'modified_activities_classifications.yaml'

# Re-export the modified data to a new YAML file
with open(modified_file_path, 'w') as modified_file:
    yaml.dump(modified_data, modified_file)